In [ ]:
!pip install torchtext==0.8.1

In [ ]:
!pip list|grep spacy

In [ ]:
import json
new_f = open("data.json", "w")
with open('latest.json', 'r') as old_f:
  count = 0
  for line in old_f:
    if "['\"" in line:
      line = line.replace("['\"", "[\"")
    if "['" in line:
      line = line.replace("['", "[\"")
    if "\"']" in line:
      line = line.replace("\"']", "\"]")
    if "']" in line:
      line = line.replace("']", "\"]")
    if "\"\"" in line:
      line = line.replace("\"\"", "")
    if "\\n" in line:
      line = line.replace("\\n", " ")
    if "\\" in line:
      line = line.replace("\\", "")
    if "\\'" in line:
      line = line.replace("\\'", "'")
    dictionary = json.loads(line)
    new_list = dictionary["sentence"][0].split(" ")
    new_list.pop()
    dictionary["sentence"] = new_list
    json_object = json.dumps(dictionary, indent = None)
    if count == 0:
      pass
    else:
      new_f.write("\n")
    count += 1
    #print(json_object)
    new_f.write(json_object)
print(count)

In [ ]:
from torchtext import data
import torch
import random

MAX_VOCAB_SIZE = 25_000
SEED = 1234
BATCH_SIZE = 5

SENTENCE = data.Field(tokenize='spacy', include_lengths=True)
CENSOR = data.LabelField()

train_data, test_data = data.TabularDataset.splits(path='',
                                                   train='data.json',
                                                   test='data.json',
                                                   fields={'sentence': ('t', SENTENCE), 'censor': ('l', CENSOR)},
                                                   format="json")

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

device = torch.device('cuda')

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data),
                                                               batch_size=BATCH_SIZE,
                                                               device=device,
                                                               sort_key=lambda x: len(x.text))

SENTENCE.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

CENSOR.build_vocab(train_data)

In [ ]:
print(train_data)

In [ ]:
print(vars(train_data[0]))

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data),
    sort_key=lambda x: len(x.t),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim*2, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(SENTENCE.vocab)

# dummy variables
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = SENTENCE.vocab.stoi[SENTENCE.pad_token]

model = Model(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

#add PAD_IDX to model as last layer

In [ ]:
pretrained_embeddings = SENTENCE.vocab.vectors

print(pretrained_embeddings.shape)

In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
from torch.autograd import Variable

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        text, text_lengths = batch.t

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.l.type_as(predictions))

        acc = binary_accuracy(predictions,batch.l.type_as(predictions))

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text, text_lengths = batch.t

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.l.type_as(predictions))

            acc = binary_accuracy(predictions, batch.l.type_as(predictions))

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_censor(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [SENTENCE.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = (torch.sigmoid(model(tensor, length_tensor)))
    return prediction.item()

In [ ]:
pred = predict_censor(model, "hell")
pred = pred*100
pred = round(pred, 1)

print("Here is my prediction: " + str(pred) + "%")